In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization, Flatten, Lambda
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.initializers import he_normal
from keras import backend as K

from IPython.display import clear_output

In [ ]:
# first let's make a directory to extract the data
!mkdir /root/data

In [ ]:
# extract the data in the created directory
!unzip ../input/training.zip -d /root/data
!unzip ../input/test.zip -d /root/data

In [ ]:
train_df = pd.read_csv('/root/data/training.csv')

In [ ]:
train_df.shape

In [ ]:
# let's see what we are working with here
train_df.head(5)

The image is stored in a list as the final column of each entry. To be able to use it as an image we need to convert it to a proper ndarray and to do that we'll use this next function

In [ ]:
def list2img(df_data):
    """
    converts img from the provided list type to a proper ndarray
    :param df_data: the image in its list type
    :return: the image in a proped ndarray shape
    """
    int_arr = np.array(df_data.split(' ')).astype(int)
    img = int_arr.reshape(96, 96)
    return img

# let's check a random image from the dataframe to check that it works as needed
rnd_idx = np.random.randint(0, train_df.shape[0])  # select a random idx
sample = train_df.Image.iloc[rnd_idx]  # select the corresponding row and get its image
sample_img = list2img(sample)  # convert it to an ndarray

plt.imshow(sample_img, cmap='gray')

Since the function works as wanted, let's apply it to all the rows of the dataframe so that we don't have to worry about the images shape later on

In [ ]:
train_df.Image = train_df.Image.apply(lambda x: list2img(x));  # apply the function to all image entries

In [ ]:
# let's check that the dataframe images column is now converted to the required shape
plt.imshow(train_df.Image.iloc[rnd_idx], cmap='gray')

Now that we have converted the images in the dataframe to ndarrays, we'll then plot the keypoints from rows onto the image to understand the data better

In [ ]:
def plot_keypoints(df_row, plot_img=True, ax=None, color='blue'):
    """
    :param df_row: a row of the dataframe with the image entrey converted
    :param plot_img: boolean for plotting the image as well or just the keypoints
    :param ax: provided axes to plot on - the function will create a new one if not provided
    :param color: color of th plotted keypoints
    :return: the axes that was used in plotting 
    """
    if not ax:
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    if plot_img:
        ax.imshow(df_row[-1], cmap='gray')
    
    for i in range(0, (df_row.size-1), 2):  # loop on every two entries since data is stored as (x, y)
        # get the x and y coordinates for each of the 15 features
        x_coord = df_row[i]
        y_coord = df_row[i+1]
        if not np.isnan(x_coord) and not np.isnan(y_coord):  # check if th coordinates aren't missing
            circle = plt.Circle((x_coord, y_coord), 1, color=color)  # create circle
            ax.add_artist(circle)  # add the circle to the axes
    return ax

rnd_idx = np.random.randint(0, train_df.shape[0])
print(train_df.iloc[rnd_idx][:-1])
plot_keypoints(train_df.iloc[rnd_idx])

Now that we have the functions we need to preprocess and visualize the data, let's check the null valus or missing coordinates in the dataset

In [ ]:
train_df.isna().sum()

### That's a lot of missing data
### we can try to handle them by:
* forward fill or back fill but both would result into a highly mislabelled dataset and the model won't have any chance to predict the test data correctly
* Use the average of each column/coordinate to fill the missing value which will make the dataset very biased towards certain values, hence the model will just to learn to predict the average most times with slight adjustments and that would actually give an OK loss on the test set since the faces are mostly cropped and centered but we'd be lying to ourselves thinking that it has learnt latent features from the data

### So we'll just have to eleminate the entries with missing values and work with very small dataset, then we can just use transfer learning and train the last few dense layers or choose a very small model so that it gives a good validation error and doesn't overfit

In [ ]:
# fill_vals = train_df.iloc[:, :-1].mean().to_dict()
# fill_vals

In [ ]:
# train_df.fillna(value=fill_vals, axis=0, inplace=True)
# train_df.fillna(method='ffill', axis=0, inplace=True)
train_df.dropna(axis=0, how='any', inplace=True)  # drop the rows with missing values

In [ ]:
train_df.shape

In [ ]:
train_df.isna().sum()  # we no longer have missing vlues

In [ ]:
faces = train_df.Image.values  # let's extract the images on their own outside the dataframe
keypoints = np.array(train_df.iloc[:, :-1])  # extract the keypoints
print(faces.shape, keypoints.shape)

In [ ]:
# faces are now an array of array we'll convert it to a 2d array
faces = np.apply_along_axis(lambda x: x.tolist(), axis=0, arr=faces)  # convert to 2d array 
faces = faces.reshape(*faces.shape, 1)  # add a dimension for color channels because convolution layers require it
faces.shape

In [ ]:
faces = faces.astype(float) / 255  # normalize the images

In [ ]:
fit_x, test_x, fit_y, test_y = train_test_split(faces, keypoints, test_size=0.1)
train_x, valid_x, train_y, valid_y = train_test_split(fit_x, fit_y, test_size=0.2)
print(f'training set shape (x, y): {train_x.shape, train_y.shape}')
print(f'validation set shape (x, y): {valid_x.shape, valid_y.shape}')
print(f'test set shape (x, y): {test_x.shape, test_y.shape}')

In [ ]:
# because we defined plot_keypoints to take a row of the dataframe as an input we use this function to combine the
# image with its keypoints as a row before sending it to the function
def wrap_as_row(keypoints, img):
    return np.array(list(keypoints) + [img])

In [ ]:
# used to plot training curves (accuracy, loss) while model is training
class Plotter(Callback):
    def plot(self):  # Updates the graph
        clear_output(wait=True)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # plot the losses
        ax1.plot(self.epochs, self.losses, label='train_loss')
        ax1.plot(self.epochs, self.val_losses, label='val_loss')
        
        # plot the accuracies
        ax2.plot(self.epochs, self.acc, label='train_acc')
        ax2.plot(self.epochs, self.val_acc, label='val_acc')
    
        ax1.set_title(f'Loss vs Epochs')
        ax1.set_xlabel("Epochs")
        ax1.set_ylabel("Loss")
        
        ax2.set_title(f'Accuracy vs Epochs')
        ax2.set_xlabel("Epoches")
        ax2.set_ylabel("Accuracy")
        
        ax1.legend()
        ax2.legend()
        plt.show()
        
        # print out the accuracies at each epoch
        print(f'Epoch #{self.epochs[-1]+1} >> train_acc={self.acc[-1]*100:.3f}%, train_loss={self.losses[-1]:.5f}')
        print(f'Epoch #{self.epochs[-1]+1} >> val_acc={self.val_acc[-1]*100:.3f}%, val_loss={self.val_losses[-1]:.5f}')
        
    def on_train_begin(self, logs={}):
        # initialize lists to store values from training
        self.losses = []
        self.val_losses = []
        self.epochs = []
        self.batch_no = []
        self.acc = []
        self.val_acc = []
    
    def on_epoch_end(self, epoch, logs={}):
        # append values from the last epoch
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.epochs.append(epoch)
        self.plot()  # update the graph
        
    def on_train_end(self, logs={}):
        self.plot()
               
plotter = Plotter()

In [ ]:
# used to decrease the learning rate if val_acc doesn't enhance
plateau_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=1, min_lr=1e-25)

In [ ]:
# not used to stop the training but just to rollback to the best weights encountered during training
e_stop = EarlyStopping(monitor='val_loss', patience=15, mode='min', restore_best_weights=True)

In [ ]:
callbacks = [plotter, plateau_reduce, e_stop]

In [ ]:
# a Convolution block with optional pooling
def conv_block(x, filters, kernel_size, strides, layer_no, add_pool=False):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, name=f'conv{layer_no}', 
               padding='same', kernel_initializer=he_normal(layer_no))(x)
    x = Activation('relu', name=f'activation{layer_no}')(x)
    x = BatchNormalization(name=f'bn{layer_no}')(x)
    if add_pool:
        x = MaxPooling2D(pool_size=[2, 2], strides=[2, 2], padding='same')(x)
    return x

In [ ]:
# a Fully connected layer with activation, batchnorm and dropout
def dense_block(x, neurons, layer_no):
    x = Dense(neurons, kernel_initializer=he_normal(layer_no), name=f'topDense{layer_no}')(x)
    x = Activation('relu', name=f'Relu{layer_no}')(x)
    x = BatchNormalization(name=f'BatchNorm{layer_no}')(x)
    x = Dropout(0.5, name=f'Dropout{layer_no}')(x)
    return x

## I found that using an extremely small model performs a bit better that using pre-trained VGG16 on imagenet with a couple of fully connected layers on top so that is what we'll be using

In [ ]:
def create_model(shape):
    input_layer = Input(shape, name='input_layer')  # input layer with given shape
    
    conv1 = conv_block(input_layer, filters=32, kernel_size=[6, 6], strides=[2, 2], layer_no=1)
    conv2 = conv_block(conv1, filters=64, kernel_size=[6, 6], strides=[3, 3], layer_no=2)
    conv3 = conv_block(conv2, filters=128, kernel_size=[6, 6], strides=[3, 3], layer_no=3)
   
    flat1 = Flatten(name='Flatten1')(conv3)
     
    output_layer = Dense(30, name='Dense1')(flat1)
    
    model = Model(inputs=[input_layer], outputs=[output_layer])

    return model

In [ ]:
# hyperparameters
height, width, channels_num = 96, 96, 1
learning_rate = 0.01
epochs = 40
batch_size = 32

In [ ]:
# define the proper loss function - same function used for evaluation
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [ ]:
model = create_model((height, width, channels_num))
optimizer = Adam(learning_rate)

model.compile(optimizer=optimizer, loss=rmse, metrics=['acc'])
model.summary()

In [ ]:
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epochs, verbose=1, 
          callbacks=callbacks, validation_data=(valid_x, valid_y), shuffle=True)

In [ ]:
metrics = model.evaluate(test_x, test_y)  # evaluate on the labelled test set we kept aside early on

In [ ]:
m_names = model.metrics_names
print(f'{m_names[0]} = {metrics[0]}\n{m_names[1]} = {metrics[1]}')

## Not bad we did next to nothing in terms of preprocessing, we could have done a lot more, and ended up with pretty decent accuray on a very dirty dataset with mislabelled, missing values, and image sharpeness variant dataset. Not bad at all!

In [ ]:
preds = model.predict(test_x)  # let's predict the same labelled test data

let's plot the model predictions against the real keypoins and see how it holds up

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10))
axs = axs.ravel()  # flatten the axs array from 2d to 1d 

for ax in axs:
    rnd_idx = np.random.randint(0, test_x.shape[0])  # select a random image
    img = test_x[rnd_idx].reshape(96, 96)
    pred = preds[rnd_idx]  # get the corresponding prediction
    true_label = test_y[rnd_idx]  # get the corresponding label
    
    pred_row = wrap_as_row(pred, img)  # wrap them together as if they are a row from the dataframe
    true_row = wrap_as_row(true_label, img) 
    ax = plot_keypoints(pred_row, plot_img=True, ax=ax, color='red')  # plot the prediction as red
    plot_keypoints(true_row, plot_img=False, ax=ax, color='blue')  # plot the true keypoints as blue

### Given the nature of the datsaset and that we didn't really preprocess the data that much, the model is performing well, but could have been better.

Now let's load the csv files for submission and see their format and how to format our submission in the same way

In [ ]:
test_df = pd.read_csv('/root/data/test.csv')
id_df = pd.read_csv('../input/IdLookupTable.csv')
sample_df = pd.read_csv('../input/SampleSubmission.csv')

In [ ]:
test_df.head(5)

In [ ]:
id_df.head(5)

In [ ]:
sample_df.head(5) 

In [ ]:
test_df.shape

In [ ]:
# let's transform the test images into an ndarray just like we did with the training images
test_df.Image = test_df.Image.apply(list2img)
test_img = test_df.Image.values
test_img = np.apply_along_axis(lambda x: x.tolist(), axis=0, arr=test_img)
test_img.shape

In [ ]:
# normalize the test images just like the training images used in training the model
test_img = test_img.astype(float) / 255

In [ ]:
test_preds = model.predict(test_img.reshape(*test_img.shape, 1))  # our predictions

### let's see some of the model predictions on the unlabelled data

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10))
axs = axs.ravel()

for i in range(axs.size):
    rnd_idx = np.random.randint(0, test_img.shape[0])

    img = test_img[rnd_idx].reshape(96, 96)
    pred = test_preds[rnd_idx]
    pred_row = wrap_as_row(pred, img)

    plot_keypoints(pred_row, plot_img=True, ax=axs[i], color='red')

### Now let's save the predictions in the required format

In [ ]:
# create a dataframe using the prediction because we'll be accessing certain features by feature/column name to
# make our submission file so this way it'll be easier
preds_df = pd.DataFrame(columns=train_df.columns[:-1], data=test_preds)

In [ ]:
preds_df.head(5)

In [ ]:
id_df.shape

In [ ]:
# create an empty dataframe with the required columns
submission = pd.DataFrame(columns=sample_df.columns, data=[])
submission.head()

In [ ]:
# iterate over the ids and get the name of the required feature name at each row
for columns, (row_id, img_id, feature, location) in id_df.iterrows():
    submission.loc[row_id-1] = [row_id, preds_df[feature].iloc[img_id-1]]

In [ ]:
submission.RowId = submission.RowId.astype(int)
submission.head(5)

In [ ]:
submission.shape

In [ ]:
id_df.head(5)  # check the format before saving

In [0]:
submission.to_csv('submission.csv', index=False)  # save to csv file

### We could have done so much better by preproessing the data more.
### One way is by using other pre-trained models to to extract useful information from the image as bounding boxes for faces or segmentation results and give it as an input to the model.

### But since the dataset contatin a lot of missing and mislabelled keypoints, and its images are variant in sharpness and have very low resolution, I would perfer to save the time of doing that on larger RGB images dataset and deploy them outside of the competition.